# 1. Write a python program which searches all the product under a particular product from www.amazon.in. The product to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search for guitars

In [138]:
import selenium
import pandas as pd
from selenium import webdriver
import time
import warnings
warnings.filterwarnings("ignore") 


#Importing requests
import requests

# importing regex
import re

# Importing required Exceptions
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait

In [4]:
driver = webdriver.Chrome(r'E:/Flip Robo Internship/chromedriver_win32/chromedriver.exe')

In [5]:
url='https://www.amazon.in/'
driver.get(url)

In [6]:
def search():
    Search_tag=driver.find_element_by_id('twotabsearchtextbox')
    search_input=str(input('Type here and I will get you there   '))
    Search_tag.send_keys(search_input)
    time.sleep(5)
    search=driver.find_element_by_id('nav-search-submit-button')
    search.click()

In [8]:
search()

Type here and I will get you there   guitars


# 2. In the above question, now scrape the following details of each product listed in first 3 pages of your search results and save it in a data frame and csv. In case if any product has less than 3 pages in search results then scrape all the products available under that product name. Details to be scraped are: "Brand Name", "Name of the Product", "Price", "Return/Exchange", "Expected Delivery", "Availability" and “Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“.

In [21]:
# Finding the element for different brands url using xpath
brands_url = driver.find_elements_by_xpath("//h2[@class='a-size-mini a-spacing-none a-color-base s-line-clamp-2']//a")

# We will run a loop to iterate all the url links extracted above and extract the href inside them
Brands_Url = []
for i in range(1,5):
    try:
        for i in brands_url:
            Brands_Url.append(i.get_attribute("href"))
    except:
        Brands_Url.append("-")
              
Brands = []
Product_Name = []
Price = []
Exchange = []
Ex_Delivery = []
Available = []
Details = []

# Let's traverse all URL one by one
for i in Brands_Url:
    driver.get(i)
    driver.implicitly_wait(5)
       
    # Finding the elements for Brand names using id
    try: 
        Brands.append(driver.find_element_by_id("productTitle").text.split(' ')[0]) 
    except: 
        Brands.append('-')
    driver.implicitly_wait(5)
    
    # Finding the elements for product names using id
    try: 
        string=''
        for i in driver.find_element_by_id("productTitle").text.split(' ')[1:]:
            string=string+' '+i
        Product_Name.append(string)
    except:
        Product_Name.append('-')
    driver.implicitly_wait(5)
    
    
    # Finding the elements for prices using id
    try: 
        try: 
            Price.append(driver.find_element_by_id("priceblock_saleprice").text) 
        except:
            try: 
                Price.append(driver.find_element_by_id("priceblock_dealprice").text)
            except: 
                Price.append(driver.find_element_by_id("priceblock_ourprice").text)
    except: 
        Price.append('-')
    driver.implicitly_wait(5)
    
    # Finding the elements for return/exchange using xpath
    try: 
        exchange = driver.find_element_by_xpath("//div[@class='a-row icon-farm-wrapper']").find_elements_by_xpath(".//div")
        found=False
        for i in exchange:
            if(i.get_attribute('data-name')=='RETURNS_POLICY'):
                found=True
                Exchange.append(i.find_element_by_xpath(".//span[1]/div[2]/a").text)
        if(found==False):
            Exchange.append('-')
    except:
        Exchange.append('-')
    driver.implicitly_wait(5)
    
    # Finding the elements for expected delivery using id
    try: 
        Ex_Delivery.append(driver.find_element_by_id("ddmDeliveryMessage").find_element_by_xpath(".//b").text)
    except: 
        Ex_Delivery.append('-')
    driver.implicitly_wait(5)
    
    # Finding the elements for availability using id
    try: 
        try: 
            Available.append(driver.find_element_by_id("availability").find_element_by_xpath(".//span").text)
        except:
            Available.append(driver.find_element_by_id("deal_availability").find_element_by_xpath(".//div/span").text)
    except: 
        Available.append('-')
    driver.implicitly_wait(5)
    
    # Finding the elements for details using id
    try:
        details = [i.text.replace('\n','---') for i in driver.find_element_by_id("productDetails_techSpec_section_1").find_elements_by_xpath(".//tbody")] 
        Details.append(details[0])
    except: 
        Details.append('-')
    driver.implicitly_wait(5)  

In [22]:
# import pandas as pd
ques_2 = pd.DataFrame({})
ques_2["BRAND NAME"] = Brands
ques_2["NAME OF THE PRODUCT"] = Product_Name
ques_2["PRICE"] = Price
ques_2["RETURN/EXCHANGE"] = Exchange
ques_2["EXPECTED DELIVERY"] = Ex_Delivery
ques_2["AVAILABLILITY"] = Available
ques_2["OTHER DETAILS"] = Details
ques_2["PRODUCT URL"] = Brands_Url
ques_2.index+=1

# Details of each product in first 3 pages
ques_2

,BRAND NAME,NAME OF THE PRODUCT,PRICE,RETURN/EXCHANGE,EXPECTED DELIVERY,AVAILABLILITY,OTHER DETAILS,PRODUCT URL


In [23]:
# Let's save the dataframe in csv file
ques_2.to_csv("product_details.csv")

# 3. Write a python program to access the search bar and search button on images.google.com and scrape 10 images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’, ‘Guitar’, ‘Cakes’.

In [27]:
#Activating the chrome browser with specified url
driver = webdriver.Chrome(r'E:/Flip Robo Internship/chromedriver_win32/chromedriver.exe')
# getting images.google.com
url = "https://images.google.com/"
#Creating empty list and giving search items as list and creating loop
urls = []    
data = []
search_item = ["fruits", "cars", "Machine Learning", "Guitar", "Cakes"]
for item in search_item:
    driver.get(url)  
    time.sleep(5)
    search_bar = driver.find_element_by_tag_name("input") #Xpath for search bar
    
    search_bar.send_keys(str(item))      #sending key word for search item
    
    search_button =driver.find_element_by_xpath("//button[@class='Tg7LZd']").click() #Clicking on search button
    
    # scrolling the web page to get more images
    for _ in range(500):
        driver.execute_script("window.scrollBy(0,100)")
        
        imgs = driver.find_elements_by_xpath("//img[@class='rg_i Q4LuWd']")
    img_url = []
    for image in imgs:
        source = image.get_attribute('src')
        if source is not None:
                if(source[0:4] == 'http'):
                    img_url.append(source)
    for i in img_url[:100]:
        urls.append(i)
                    
for i in range(len(urls)):
    if i >= 300:
        break
    print("Downloading {0} of {1} images" .format(i, 300))
    response = requests.get(urls[i])

    file = open(r"E:\Flip Robo Internship\Assignments\Web Scrapping Assignment\Web Scraping Assignment-3"+str(i)+".jpg", "wb")

    file.write(response.content)

# 4. Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com and scrape following details for all the search results displayed on 1st page. Details to be scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”, “Secondary Camera”, “Display Size”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the details is missing then replace it by “- “. Save your results in a dataframe and CSV.

In [166]:
url = 'https://www.flipkart.com/'
driver = webdriver.Chrome(r'E:/Flip Robo Internship/chromedriver_win32/chromedriver.exe')
driver.get(url)
driver.maximize_window()
time.sleep(2)

In [167]:
search_product = driver.find_element_by_xpath('//input[@class="_3704LK"]')
search_btn= driver.find_element_by_xpath('//button[@class ="L0Z3Pu"]')


In [169]:
search_product.send_keys('Oneplus Nord, pixel 4A')
search_btn.click()

In [170]:
Url=[]
url = driver.find_elements_by_xpath('//a[@class ="_1fQZEK"]')
for i in url:
    Url.append(i.get_attribute('href'))

In [171]:
Brand_name =[]
Price=[]
Colour=[]
Display_size=[]
Display_resolution=[]
Primary_Camera=[]
Secondary_camera=[]
RAM=[]
ROM= []
Battery_capacity =[]
Processor =[]
Processor_core =[]

for i in Url:
    driver.get(i)
    sleep(2)
    
    readmore =driver.find_element_by_xpath('//button[@class ="_2KpZ6l _1FH0tX"]')
    readmore.click()
    sleep(2)

    try:
        brand = driver.find_element_by_xpath('//span[@class="B_NuCI"]')
        Brand_name.append(brand.text)
        
    except:
        Brand_name.append('--') 
    
    
    try:
        price = driver.find_element_by_xpath('//div[@class="_30jeq3 _16Jk6d"]')
        Price.append(price.text)
        
    except:
        Price.append('--')
        
    try:
        color = driver.find_element_by_xpath('//div [@class ="_3k-BhJ"][1]/table/tbody/tr[4]/td[2]/ul/li')
        Colour.append(color.text)
        
    except:
        Colour.append('--')
        
    try:
        dsize = driver.find_element_by_xpath('//div [@class ="_3k-BhJ"][2]/table/tbody/tr[1]/td[2]/ul/li')
        Display_size.append(dsize.text)
        
    except:
        Display_size.append('--')
        
        
    try:
        resolution = driver.find_element_by_xpath('//div [@class ="_3k-BhJ"][2]/table/tbody/tr[2]/td[2]/ul/li')
        Display_resolution.append(resolution.text)
        
    except:
        Display_resolution.append('--')
        
        
    try:
        pcamera = driver.find_element_by_xpath('//div [@class ="_3k-BhJ"][5]/table/tbody/tr[2]/td[2]/ul/li')
        Primary_Camera.append(pcamera.text)
        
    except:
        Primary_Camera.append('--')
        
        
    try:
        scamera = driver.find_element_by_xpath('//div [@class ="_3k-BhJ"][5]/table/tbody/tr[5]/td[2]/ul/li')
        Secondary_camera.append(scamera.text)
        
    except:
        Secondary_camera.append('--')
        
        
    try:
        ram = driver.find_element_by_xpath('//div [@class ="_3k-BhJ"][4]/table/tbody/tr[2]/td[2]/ul/li')
        RAM.append(ram.text)
        
    except:
        RAM.append('--')
        
        
        
    try:
        rom = driver.find_element_by_xpath('//div [@class ="_3k-BhJ"][4]/table/tbody/tr[1]/td[2]/ul/li')
        ROM.append(rom.text)
        
    except:
        ROM.append('--')
        
        
        
    try:
        battery = driver.find_element_by_xpath('//div [@class ="_3k-BhJ"][10]/table/tbody/tr[1]/td[2]/ul/li')
        Battery_capacity.append(battery.text)
        
    except:
        Battery_capacity.append('--')
        
        
    try:
        processor = driver.find_element_by_xpath('//div [@class ="_3k-BhJ"][3]/table/tbody/tr[2]/td[2]/ul/li')
        Processor.append(processor.text)
        
    except:
        Processor.append('--')
        
        
        
    try:
        core = driver.find_element_by_xpath('//div [@class ="_3k-BhJ"][3]/table/tbody/tr[3]/td[2]/ul/li')
        Processor_core.append(core.text)
        
    except:
        Processor_core.append('--')

In [172]:
Smartphone_data = pd.DataFrame({})

Smartphone_data['Brand_name / Mobile name'] =Brand_name
Smartphone_data['Price'] =Price
Smartphone_data['Colour'] =Colour
Smartphone_data['Display_size'] =Display_size
Smartphone_data['Display_resolution'] =Display_resolution
Smartphone_data['Primary_Camera'] =Primary_Camera
Smartphone_data['Secondary_camera'] =Secondary_camera
Smartphone_data['RAM'] =RAM
Smartphone_data['ROM'] =ROM
Smartphone_data['Battery_capacity'] =Battery_capacity 
Smartphone_data['Processor'] =Processor 
Smartphone_data['Processor_core'] =Processor_core 
Smartphone_data['Product URL'] = Url

Smartphone_data 

,Brand_name / Mobile name,Price,Colour,Display_size,Display_resolution,Primary_Camera,Secondary_camera,RAM,ROM,Battery_capacity,Processor,Processor_core,Product URL


# 5. Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on googlemaps.


In [39]:
driver = webdriver.Chrome(r'E:/Flip Robo Internship/chromedriver_win32/chromedriver.exe')
# Get the webpage
url="https://www.google.co.in/maps/"
driver.get(url)
driver.maximize_window()

In [40]:
search_city=driver.find_element_by_xpath("//input[@id='searchboxinput']")
search_city.send_keys('Panipat Haryana')

In [41]:
search_btn= driver.find_element_by_xpath("//button[@id='searchbox-searchbutton']")
search_btn.click()

In [42]:
coordinate=driver.find_element_by_xpath("//div[@class='gb_1f gb_h']/div/a")

In [43]:
x=coordinate.get_attribute('href')

In [44]:
cordinates= re.findall('\d\d\.\d+', x)

In [45]:
cordinates

['29.383028',
 '76.9371419',
 '13.56',
 '29.3909464',
 '76.9635023',
 '29.3909464',
 '76.9635023']

In [57]:
Longitutde =cordinates[0:4:2]

In [58]:
Longitutde

['29.383028', '13.56']

In [59]:
latitude=cordinates[1:4:2]

In [60]:
latitude

['76.9371419', '29.3909464']

In [61]:
maps=pd.DataFrame({})
maps['Longitutde']=Longitutde
maps['latitude']=latitude
maps

,Longitutde,latitude
0,29.383028,76.9371419
1,13.56,29.3909464


# 6. Write a program to scrap details of all the funding deals for second quarter (i.e Jan 21 – March 21) from trak.in.

In [66]:
driver = webdriver.Chrome(r'E:/Flip Robo Internship/chromedriver_win32/chromedriver.exe')
url='https://trak.in/'
driver.get(url)
driver.maximize_window()
time.sleep(2)

In [67]:
funding_tab = driver.find_element_by_xpath('//li[@id ="menu-item-51510"]')
funding_tab.click()

In [68]:
Date =[]
Startup_name =[]
Industry_vertical =[]
Sub_vertical =[]
City_location=[]
Investers_name=[]
Investment_type =[]
Amount =[]



dateJly =driver.find_elements_by_xpath('//div[@id ="tablepress-48_wrapper"]/div[3]/div[2]/table/tbody/tr//td[2]')
dateAug =driver.find_elements_by_xpath('//div[@id ="tablepress-49_wrapper"]/div[3]/div[2]/table/tbody/tr//td[2]')
datesep =driver.find_elements_by_xpath('//div[@id ="tablepress-50_wrapper"]/div[3]/div[2]/table/tbody/tr//td[2]')

for i in [dateJly,dateAug,datesep]:
    try:
        for j in i:
            Date.append(j.text)
    except:
        Date.append('--')

startupJly =driver.find_elements_by_xpath('//div[@id ="tablepress-48_wrapper"]/div[3]/div[2]/table/tbody/tr//td[3]')
startupAug =driver.find_elements_by_xpath('//div[@id ="tablepress-49_wrapper"]/div[3]/div[2]/table/tbody/tr//td[3]')
startupsep =driver.find_elements_by_xpath('//div[@id ="tablepress-50_wrapper"]/div[3]/div[2]/table/tbody/tr//td[3]')

for i in [startupJly,startupAug,startupsep]:
    try:
        for j in i:
            Startup_name.append(j.text)
    except:
        Startup_name.append('--')

IndustryJly =driver.find_elements_by_xpath('//div[@id ="tablepress-48_wrapper"]/div[3]/div[2]/table/tbody/tr//td[4]')
IndustryAug =driver.find_elements_by_xpath('//div[@id ="tablepress-49_wrapper"]/div[3]/div[2]/table/tbody/tr//td[4]')
Industrysep =driver.find_elements_by_xpath('//div[@id ="tablepress-50_wrapper"]/div[3]/div[2]/table/tbody/tr//td[4]')

for i in [IndustryJly,IndustryAug,Industrysep]:
    try:
        for j in i:
            Industry_vertical.append(j.text)
    except:
        Industry_vertical.append('--')

Sub_verticalJly =driver.find_elements_by_xpath('//div[@id ="tablepress-48_wrapper"]/div[3]/div[2]/table/tbody/tr//td[5]')
Sub_verticalAug =driver.find_elements_by_xpath('//div[@id ="tablepress-49_wrapper"]/div[3]/div[2]/table/tbody/tr//td[5]')
Sub_verticalsep =driver.find_elements_by_xpath('//div[@id ="tablepress-50_wrapper"]/div[3]/div[2]/table/tbody/tr//td[5]')

for i in [Sub_verticalJly,Sub_verticalAug,Sub_verticalsep]:
    try:
        for j in i:
            Sub_vertical.append(j.text)
    except:
        Sub_vertical.append('--')

CityJly =driver.find_elements_by_xpath('//div[@id ="tablepress-48_wrapper"]/div[3]/div[2]/table/tbody/tr//td[6]')
CityAug =driver.find_elements_by_xpath('//div[@id ="tablepress-49_wrapper"]/div[3]/div[2]/table/tbody/tr//td[6]')
Citysep =driver.find_elements_by_xpath('//div[@id ="tablepress-50_wrapper"]/div[3]/div[2]/table/tbody/tr//td[6]')

for i in [CityJly,CityAug,Citysep]:
    try:
        for j in i:
            City_location.append(j.text)
    except:
        City_location.append('--')

InvestersJly =driver.find_elements_by_xpath('//div[@id ="tablepress-48_wrapper"]/div[3]/div[2]/table/tbody/tr//td[7]')
InvestersAug =driver.find_elements_by_xpath('//div[@id ="tablepress-49_wrapper"]/div[3]/div[2]/table/tbody/tr//td[7]')
Investerssep =driver.find_elements_by_xpath('//div[@id ="tablepress-50_wrapper"]/div[3]/div[2]/table/tbody/tr//td[7]')

for i in [InvestersJly,InvestersAug,Investerssep]:
    try:
        for j in i:
            Investers_name.append(j.text)
    except:
        Investers_name.append('--')

Iv_typeJly =driver.find_elements_by_xpath('//div[@id ="tablepress-48_wrapper"]/div[3]/div[2]/table/tbody/tr//td[8]')
Iv_typeAug =driver.find_elements_by_xpath('//div[@id ="tablepress-49_wrapper"]/div[3]/div[2]/table/tbody/tr//td[8]')
Iv_typesep =driver.find_elements_by_xpath('//div[@id ="tablepress-50_wrapper"]/div[3]/div[2]/table/tbody/tr//td[8]')

for i in [Iv_typeJly,Iv_typeAug,Iv_typesep]:
    try:
        for j in i:
            Investment_type.append(j.text)
    except:
        Investment_type.append('--')

AmountJly =driver.find_elements_by_xpath('//div[@id ="tablepress-48_wrapper"]/div[3]/div[2]/table/tbody/tr//td[9]')
AmountAug =driver.find_elements_by_xpath('//div[@id ="tablepress-49_wrapper"]/div[3]/div[2]/table/tbody/tr//td[9]')
Amountsep =driver.find_elements_by_xpath('//div[@id ="tablepress-50_wrapper"]/div[3]/div[2]/table/tbody/tr//td[9]')

for i in [AmountJly,AmountAug,Amountsep]:
    try:
        for j in i:
            Amount.append(j.text)
    except:
        Amount.append('--')

In [69]:
Quaterly_funding_dataset =pd.DataFrame({})

Quaterly_funding_dataset['Date'] = Date
Quaterly_funding_dataset['Startup_name'] = Startup_name 
Quaterly_funding_dataset['Industry / vertical'] = Industry_vertical 
Quaterly_funding_dataset['Sub_vertical'] = Sub_vertical 
Quaterly_funding_dataset['City / location'] = City_location
Quaterly_funding_dataset["Investers'name"] = Investers_name
Quaterly_funding_dataset['Investment_type'] = Investment_type 
Quaterly_funding_dataset['Amount in(USD)'] = Amount

Quaterly_funding_dataset

,Date,Startup_name,Industry / vertical,Sub_vertical,City / location,Investers'name,Investment_type,Amount in(USD)


# 7. Write a program to scrap all the available details of best gaming laptops from digit.in.

In [70]:
driver = webdriver.Chrome(r'E:/Flip Robo Internship/chromedriver_win32/chromedriver.exe')
# Get the webpage
url="https://www.digit.in/"
driver.get(url)
driver.maximize_window()

In [71]:
Laptop_btn=driver.find_element_by_xpath("/html/body/div[1]/div/div[4]/ul/li[3]/a")
Laptop_btn.click()


In [72]:
Latest_btn=driver.find_element_by_xpath("/html/body/div[6]/div/div[2]/div[2]/ul/li[10]/a")
Latest_btn.click()

In [73]:
Product_name=[]
OS=[] 
Display=[]
Processor=[]
Memory=[]
Weight=[]
Dimension=[]
Graphics_Processor=[]

product_name= driver.find_elements_by_xpath("//div[@data-cat='Top_Ten_En_Scroll']/a/h3")
for p in product_name:
    Product_name.append(p.text)
    
Os=driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr[3]/td[3]")
for o in Os:
    OS.append(o.text)

display=driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr[4]/td[3]")
for d in display:
    Display.append(d.text)

processor=driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr[5]/td[3]")
for pr in processor:
    Processor.append(pr.text)
    
memory =driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr[6]/td[3]")
for m in memory:
    Memory.append(m.text)
    
weight =driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr[7]/td[3]")
for w in weight:
    Weight.append(w.text)
    
dimension =driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr[8]/td[3]")
for d in dimension:
    Dimension.append(d.text)

graphics_Processor=driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr[9]/td[3]")
for g in graphics_Processor:
    Graphics_Processor.append(g.text)

In [74]:
Price=[]
price=driver.find_elements_by_xpath("//table[@id='summtable']")
for i in price:
    Price.append(i.text)

In [75]:
Laptops=pd.DataFrame({})
Laptops['Product_name']=Product_name
Laptops['OS']=OS
Laptops['Display']=Display
Laptops['Memory']=Memory
Laptops['Weight']=Weight
Laptops['Dimension']=Dimension
Laptops['Graphics_Processor']=Graphics_Processor
Laptops

,Product_name,OS,Display,Memory,Weight,Dimension,Graphics_Processor
0,ACER NITRO 5 RYZEN 9 (2021),Windows 10,"15.6"" (1920 x 1080)",1 TB HDD/16 GBGB DDR4,2.4,363.4 x 255 x 23.9,NVIDIA GeForce RTX 3070
1,MSI STEALTH 15M 11TH GEN CORE I7-11375H (2021,Windows 10,"15.6"" (1920 x 1080)",1 TB SSD/16 GBGB DDR4,1.7,358.3 x 248 x 16.15,NVIDIA GeForce RTX 3060
2,ASUS ROG STRIX SCAR 15 RYZEN 9-5900HX (2021),Windows 10,"15.6"" (2560 x 1440)",2 TB SSD/32 GBGB DDR4,2.30,354 x 259 x 22.6,NVIDIA GeForce RTX 3080
3,ALIENWARE AREA 51M R2,Windows 10 Home,"17.3"" (1920 x 1080)",1 TB SSD/16 GBGB DDR4,4.1,27.65 x 402.6 x 319.14,Intel® UHD Graphics 630
4,ALIENWARE M15 R3,Windows 10 Home,"15.6"" (3840 x 2160)",1 TB SSD/16 GBGB DDR4,NA,NA,NA
5,ASUS ROG STRIX SCAR 15,Windows 10 Home,"15.6"" (1920 x 1080)",1 TB SSD/16 GBGB DDR4,2.30,35.4 x 25.9 x 2.26,NVIDIA® GeForce RTX™ 3070
6,ASUS ROG ZEPHYRUS G14,Windows 10 Home,"14"" (1920 x 1080)",1 TB SSD/16 GBGB DDR4,1.65,32.5 x 22.1 x 1.8,NVIDIA GeForce RTX 2060
7,LENOVO LEGION 5I,Windows 10 Pro,"15.6"" (1920 x 1080)",1 TB SSD/16 GBGB DDR4,2.3,363.06 x 259.61 x 23.57,NVIDIA® GeForce® GTX 1650 4GB
8,ASUS ROG ZEPHYRUS DUO 15,Windows 10,"15.6"" (3840 x 1100)",512 GB SSD/4 GBGB DDR4,2.4,268.30 x 360.00 x 20.90,NVIDIA GeForce RTX 2070 Max-Q
9,ACER ASPIRE 7 GAMING,Windows 10 Home,"15.6"" (1920 x 1080)",512 GB SSD/8 GBGB DDR4,2.15,2.29 x 36.3 x 25.4,NVIDIA® GeForce® GTX 1650


# 8. Write a python program to scrape the details for all billionaires from www.forbes.com. Details to be scrapped: “Rank”, “Name”, “Net worth”, “Age”, “Citizenship”, “Source”, “Industry”.


In [76]:
driver = webdriver.Chrome(r'E:/Flip Robo Internship/chromedriver_win32/chromedriver.exe')
# Get the webpage
url='https://www.forbes.com/billionaires/'
driver.get(url)
driver.maximize_window()

In [78]:
Rank=[]
Name=[]
Net_worth=[]
Age=[]
Citizenship=[]
Source=[]
Industry =[]
for i in range(14):
    print('Scraping Page',i+1)
    
    rank=driver.find_elements_by_xpath("//div[@class='rank']")
    for r in rank:
        Rank.append(r.text)  
    time.sleep(1) 
    
    name=driver.find_elements_by_xpath("//div[@class='personName']")
    for n in name:
        Name.append(n.text)
    time.sleep(1)
    
    net_worth=driver.find_elements_by_xpath("//div[@class='netWorth']")
    for nw in net_worth:
        Net_worth.append(nw.text)
    time.sleep(1)
    
    age=driver.find_elements_by_xpath("//div[@class='age']")
    for a in age:
        Age.append(a.text)
    time.sleep(1)
        
    citizenship=driver.find_elements_by_xpath("//div[@class='countryOfCitizenship']")
    for c in citizenship:
        Citizenship.append(c.text)
    time.sleep(1)
        
    source=driver.find_elements_by_xpath("//div[@class='source-column']")
    for s in source:
        Source.append(s.text)
    time.sleep(1)
        
    industry=driver.find_elements_by_xpath("//div[@class='category']")
    for i in industry:
        Industry.append(i.text)
    time.sleep(1)
    try:
        Next_btn=driver.find_element_by_xpath("//div[@class='next-page']")
        Next_btn.click()
        time.sleep(2)
    except:
        break             

Scraping Page 1
Scraping Page 2
Scraping Page 3
Scraping Page 4
Scraping Page 5
Scraping Page 6
Scraping Page 7
Scraping Page 8
Scraping Page 9
Scraping Page 10
Scraping Page 11
Scraping Page 12
Scraping Page 13
Scraping Page 14


In [79]:
Billionaires=pd.DataFrame({})
Billionaires['Rank']=Rank
Billionaires['Name']= Name
Billionaires['Net_worth']=Net_worth
Billionaires['Age']=Age
Billionaires['Citizenship']=Citizenship
Billionaires['Source']=Source
Billionaires['Industry']=Industry
Billionaires

,Rank,Name,Net_worth,Age,Citizenship,Source,Industry
0,1.,Jeff Bezos,$177 B,57,United States,Amazon,Technology
1,2.,Elon Musk,$151 B,49,United States,"Tesla, SpaceX",Automotive
2,3.,Bernard Arnault & family,$150 B,72,France,LVMH,Fashion & Retail
3,4.,Bill Gates,$124 B,65,United States,Microsoft,Technology
4,5.,Mark Zuckerberg,$97 B,36,United States,Facebook,Technology
...,...,...,...,...,...,...,...
2750,2674.,Daniel Yong Zhang,$1 B,49,China,e-commerce,Technology
2751,2674.,Zhang Yuqiang,$1 B,65,China,Fiberglass,Manufacturing
2752,2674.,Zhao Meiguang,$1 B,58,China,gold mining,Metals & Mining
2753,2674.,Zhong Naixiong,$1 B,58,China,conglomerate,Diversified


# 9. Write a program to extract at least 500 Comments, Comment upvote and time when comment was posted from any YouTube Video.

In [149]:
# Activating the chrome browser
driver = webdriver.Chrome(r'E:/Flip Robo Internship/chromedriver_win32/chromedriver.exe')

# Opening the youtube.com
url = "https://www.youtube.com/"
driver.get(url)
time.sleep(2)

i=0
while(i<100):
    driver.execute_script("window.scrollBy(0,500)") # scroll down to get more comments
    i+=1
while(i<402):
    driver.execute_script("window.scrollBy(0,5000)") # scroll down to get more comments
    i+=1

In [150]:
cmt = []
upvt = []
cmttime = []

In [151]:
cmt_tag = driver.find_elements_by_xpath("//*[@id='content-text']")
for i in cmt_tag:
    cmt.append(i.text.replace("\n",''))

upvt_tag = driver.find_elements_by_xpath("//*[@id='vote-count-middle']")
for j in upvt_tag:
    try: 
        upvt.append(j.text)
    except:
        upvt.append('-')
cmttime_tag = driver.find_elements_by_xpath("//*[@id='header-author']/yt-formatted-string/a")
for k in cmttime_tag:
    try: 
        cmttime.append(k.text)
    except:
        cmttime.append('-')

In [152]:
for i in range(0,len(upvt)):
    if upvt[i] == '': 
        upvt[i] = '-' # replacing empty string with '-'

In [153]:
Youtube = {'Comment':cmt,'Commented':cmttime,'Number of upvotes':upvt}

In [154]:
YoutubeDF = pd.DataFrame(Youtube)
YoutubeDF

,Comment,Commented,Number of upvotes


# 10. Write a python program to scrape a data for all available Hostels from https://www.hostelworld.com/ in “London” location. You have to scrape hostel name, distance from city centre, ratings, total reviews, overall reviews, privates from price, dorms from price, facilities and property description.

In [107]:
driver = webdriver.Chrome(r'E:/Flip Robo Internship/chromedriver_win32/chromedriver.exe')
url="https://www.hostelworld.com/"
driver.get(url)
driver.maximize_window()

In [109]:
search_hotel=driver.find_element_by_xpath("//input[@id='search-input-field']")
search_hotel.click()
time.sleep(2)


In [110]:
search_hotel1=driver.find_element_by_xpath("//input[@id='search-input-field']")
search_hotel1.send_keys('London')
time.sleep(2)

In [111]:
search_btn1=driver.find_element_by_xpath("//div[@class='label']")
search_btn1.click()
time.sleep(2)

In [112]:
search_btn=driver.find_element_by_xpath("//button[@id='search-button']")
search_btn.click()
time.sleep(2)

In [113]:
Hotel_Name=[]
Dist_from_city=[]
Total_reviews=[]
Overall_reviews=[]
Facility=[]
Page_url=[]


for i in range(3):
    print('Scraping Page',i+1)
    
    hotel_name=driver.find_elements_by_xpath("//h2[@class='title title-6']")
    for h in hotel_name:
        Hotel_Name.append(h.text)
        
    page_url=driver.find_elements_by_xpath("//h2[@class='title title-6']/a")
    for r in page_url:
        Page_url.append(r.get_attribute('href'))
    
    dist_from_city=driver.find_elements_by_xpath("//span[@class='description']")
    for dist in dist_from_city:
        Dist_from_city.append(dist.text)
    
    total_reviews=driver.find_elements_by_xpath("//div[@class='reviews']")
    for t in total_reviews:
        Total_reviews.append(t.text.replace('Total Reviews','').strip())
        
    overall_reviews=driver.find_elements_by_xpath("//div[@class='keyword']")
    for t in overall_reviews:
        Overall_reviews.append(t.text)
        
  
    facility=driver.find_elements_by_xpath("//div[@class='facilities-label facilities']")
    for t in facility:
        Facility.append(t.text.replace('\n','--'))
    try:
        Next_btn=driver.find_element_by_xpath("//div[@class='pagination-item pagination-next']")
        Next_btn.click() 
        time.sleep(2)
    except:
        break

Scraping Page 1


In [115]:
Ratings=[]
Property_desc=[]
for i in Page_url:
    driver.get(i)
    time.sleep(2)
    try:
        property_desc=driver.find_element_by_xpath("//div[@class='content collapse-content']")
        Property_desc.append(property_desc.text)
    except:
        Property_desc.append('--')
    
    try:
        rating=driver.find_element_by_xpath("//div[@class='score orange big' or @class='score gray big']")
        Ratings.append(rating.text)
    except:
        Ratings.append('--')

In [117]:
Hotel_detail=pd.DataFrame({})
Hotel_detail['Hotel_Name']= Hotel_Name
Hotel_detail['Dist_from_city']=Dist_from_city
Hotel_detail['Total_reviews']=Total_reviews
Hotel_detail['Overall_reviews']=Overall_reviews
Hotel_detail['Facility']=Facility

Hotel_detail

,Hotel_Name,Dist_from_city,Total_reviews,Overall_reviews,Facility
